# The Countdown Show, Numbers-Game 
Countdown is a british show that broadcast on Channel 4. In the show, there are three games that are played in the show. We'll be concerned with the game that is called "Countdown" - Essentially, the contestants must use arithmetic mathematics to reach a random target number from six other numbers.

We'll be using the code available to us from the college labs, except the code will be refactored to use the functional approach.

In [24]:
# Import all the libraries that the lab used.
import itertools as it
import random
import operator
import pandas as pd

# Generate the list of the "large" numbers.
def generate_large(step = 25, amount = 4):
	res = []

	for i in range(1, amount + 1):
		res.append( i * step )

	return res

# Generating the small list, we'll use the same function we used to get the large numbers, except we'll manipulate (order and double the amount of them, not the value) the numbers
def generate_small():
	return sorted( generate_large(step = 1, amount = 10) * 2 )


In [25]:
# Generate large numbers
large_numbers = generate_large(25, 4)
# Generate small numbers
small_numbers = generate_small()

# Fit that data into a pandas data frame
df = pd.DataFrame(large_numbers, columns = ['Large'])
# And finally display it
df

,Large
0,25
1,50
2,75
3,100


In [26]:
# Peek the small numbers as well, just to make sure we're correct.
df = pd.DataFrame(small_numbers, columns = ['Small'])
df

,Small
0,1
1,1
2,2
3,2
4,3
5,3
6,4
7,4
8,5
9,5
